## 目录
- [（关联的两个概念中，一个有对应的ICD编码，另一个可以在 Snomed CT 中查到 Description）的关系](#（关联的两个概念中，一个有对应的ICD编码，另一个可以在-Snomed-CT-中查到-Description）的关系)
- [两个概念均有对应的ICD编码 的关系](#两个概念均有对应的ICD编码的关系)

In [1]:
import pandas as pd
import pymongo
import re 
import random
from copy import deepcopy
import requests, json

In [30]:
icd2sct = pd.read_table('Download/Mapping/mapSnomedToIcd.txt')

In [31]:
icd2sct

,conceptId,亚目编码,mapAdvice,term,亚目名称
0,240349003,A00.0,ALWAYS A00.0,Cholera due to Vibrio cholerae O1 Classical bi...,"霍乱,由于01群霍乱弧菌,霍乱生物型所致"
1,240349003,A00.0,ALWAYS A00.0,Cholera caused by Vibrio cholerae O1 Classical...,"霍乱,由于01群霍乱弧菌,霍乱生物型所致"
2,240349003,A00.0,ALWAYS A00.0,Cholera caused by Vibrio cholerae O1 Classical...,"霍乱,由于01群霍乱弧菌,霍乱生物型所致"
3,81020007,A00.1,ALWAYS A00.1,Cholera due to Vibrio cholerae El Tor,"霍乱,由于01群霍乱弧菌,埃尔托生物型所致"
4,81020007,A00.1,ALWAYS A00.1,Cholera - Vibrio cholerae O1 El Tor biotype,"霍乱,由于01群霍乱弧菌,埃尔托生物型所致"
5,81020007,A00.1,ALWAYS A00.1,Cholera caused by Vibrio cholerae El Tor,"霍乱,由于01群霍乱弧菌,埃尔托生物型所致"
6,81020007,A00.1,ALWAYS A00.1,Cholera caused by Vibrio cholerae El Tor (diso...,"霍乱,由于01群霍乱弧菌,埃尔托生物型所致"
7,63650001,A00.9,ALWAYS A00.9,Cholera,未特指的霍乱
8,63650001,A00.9,ALWAYS A00.9,Cholera (disorder),未特指的霍乱
9,63650001,A00.9,ALWAYS A00.9,Vibrio cholerae infection,未特指的霍乱


In [5]:
for line in list(icd2sct['conceptId']):
    assert type(line) == int

In [44]:
rela = pd.read_table('Download/SnomedCT_InternationalRF2_PRODUCTION_20190731T120000Z/Delta/Terminology/sct2_Relationship_Delta_INT_20190731.txt')

In [7]:
rela = rela[rela['active']==1]

### （关联的两个概念中，一个有对应的ICD编码，另一个可以在 Snomed CT 中查到 Description）的关系

In [8]:
client = pymongo.MongoClient(host='localhost', port=27017)
db = client.sym_info.sym_info

In [9]:
sym_sctid_list = [re.findall(r'sctid-(\d+)', item['bmesh'])[0] for item in db.find() if len(re.findall(r'sctid-(\d+)', item['bmesh']))>0]

In [10]:
set(icd2sct['conceptId']).intersection(set(sym_sctid_list))

set()

In [55]:
icd_id_set = set(icd2sct['conceptId'])

In [ ]:
sym_id_set = set(sym_sctid_list)

In [12]:
descri = pd.read_table('Download/SnomedCT_InternationalRF2_PRODUCTION_20190731T120000Z/Delta/Terminology/sct2_Description_Delta-en_INT_20190731.txt')

In [13]:
descri = descri[descri['active']==1]

In [16]:
for row in rela.iterrows():
    sctids = tuple(row[1][['sourceId', 'destinationId']])
    if sctids[0] in icd_id_set and len(descri[descri['conceptId']==sctids[1]]) != 0:
        print('sctids[0] in icd_id_set')
        print(icd2sct[icd2sct['conceptId']==sctids[0]])
        print('-----------------------------')
        print(descri[descri['conceptId']==sctids[1]])
        print(' *****************************\n *****************************\n *****************************\n')
        break
    elif sctids[1] in icd_id_set and len(descri[descri['conceptId']==sctids[0]]) != 0:
        print('sctids[1] in icd_id_set')
        print(icd2sct[icd2sct['conceptId']==sctids[1]])
        print('-----------------------------')
        print(descri[descri['conceptId']==sctids[0]])
        print(' *****************************\n *****************************\n *****************************\n')

sctids[1] in icd_id_set
        conceptId   亚目编码                                          mapAdvice  \
195302  129135003  T11.3  ALWAYS T11.3 | POSSIBLE REQUIREMENT FOR AN EXT...   
195303  129135003  T11.3  ALWAYS T11.3 | POSSIBLE REQUIREMENT FOR AN EXT...   

                                                 term              亚目名称  
195302             Injury of nerve of upper extremity  上肢未特指神经的损伤,水平未特指  
195303  Injury of nerve of upper extremity (disorder)  上肢未特指神经的损伤,水平未特指  
-----------------------------
               id  effectiveTime  active            moduleId  conceptId  \
12385  3767137017       20190731       1  900000000000207008  212295006   
12386  3767138010       20190731       1  900000000000207008  212295006   

      languageCode              typeId  \
12385           en  900000000000003001   
12386           en  900000000000013009   

                                                term  caseSignificanceId  
12385  Injury of intercostobrachial nerve (disorder)  9000

In [42]:
data = []
for row in rela.iterrows():
    row_result = {}
    sctids = tuple(row[1][['sourceId', 'destinationId']])
    if sctids[0] in icd_id_set and len(descri[descri['conceptId']==sctids[1]]) != 0:
        icd_result = icd2sct[icd2sct['conceptId']==sctids[0]]
        row_result['icd_id'] = list(icd_result['亚目编码'])[0]
        row_result['icd_name'] = list(icd_result['亚目名称'])[0]
        row_result['icd_where'] = 'source'
        row_result['another_sctid'] = deepcopy(sctids[1])
        descri_result = descri[descri['conceptId']==sctids[1]]
        row_result['another_sct_term'] = list(descri_result['term'])[0]
        row_result['relation_type_id'] = row[1]['typeId']
        data.append(deepcopy(row_result))
    
    elif sctids[1] in icd_id_set and len(descri[descri['conceptId']==sctids[0]]) != 0:
        icd_result = icd2sct[icd2sct['conceptId']==sctids[1]]
        row_result['icd_id'] = list(icd_result['亚目编码'])[0]
        row_result['icd_name'] = list(icd_result['亚目名称'])[0]
        row_result['icd_where'] = 'destination'
        row_result['another_sctid'] = deepcopy(sctids[0])
        descri_result = descri[descri['conceptId']==sctids[0]]
        row_result['another_sct_term'] = list(descri_result['term'])[0]
        row_result['relation_type_id'] = row[1]['typeId']
        data.append(deepcopy(row_result))
        

In [46]:
data_df = pd.DataFrame(data)[['icd_id', 'icd_name', 'icd_where', 'another_sctid', 'another_sct_term', 'relation_type_id', ]]

#### 翻译 sct term

In [60]:
def caiyunQuery(q):
    url = "http://api.interpreter.caiyunai.com/v1/translator"
 
    token = "mql8jp2ciiq0mqjqbrmo"

    payload = {
            "source" : q, 
            "trans_type" : "en2zh",
            "request_id" : "demo",
            }

    headers = {
            'content-type': "application/json",
            'x-authorization': "token " + token,
    }

    response = requests.request("POST", url, data=json.dumps(payload), headers=headers)
    #print(response.text[:2500])

    return json.loads(response.text)

In [61]:
caiyunResult = []
for i in range(4604//50+1):
    temp = list(data_df['another_sct_term'])
    res = caiyunQuery(temp[i*50:(i+1)*50])
    caiyunResult += res['target']

In [63]:
assert len(caiyunResult) == 4605 

In [64]:
data_df['another_sct_term_cn'] = caiyunResult

In [65]:
data_df

,icd_id,icd_name,icd_where,another_sctid,another_sct_term,relation_type_id,another_sct_term_cn
0,T11.3,"上肢未特指神经的损伤,水平未特指",destination,212295006,Injury of intercostobrachial nerve (disorder),116680003,肋间神经损伤(失调)
1,R04.0,鼻出血,source,95433000,Disorder of nasal septum (disorder),116680003,鼻中隔紊乱
2,Q31.9,喉未特指的先天性畸形,destination,232461002,Congenital fissure of larynx,116680003,先天性喉裂
3,K26.9,"十二指肠溃疡未特指为急性或慢性,不伴有出血或穿孔",source,196652006,Acute peptic ulcer of duodenum,116680003,急性十二指肠消化性溃疡
4,L98.9,皮肤和皮下组织未特指的疾患,destination,238561006,Chronic vesicular eczema of foot,116680003,足部慢性水疱性湿疹
5,G71.0,肌营养不良,source,240046001,Limb-girdle muscular dystrophy,116680003,肢节型肌营养不良症
6,S09.9,头部未特指的损伤,source,95433000,Disorder of nasal septum (disorder),116680003,鼻中隔紊乱
7,T94.1,"损伤后遗症,未按身体部位特指者",destination,285236005,Disorder due to and following injury of upper ...,116680003,上肢损伤(障碍)所致或继发的功能障碍
8,D10.1,舌良性肿瘤,source,91975001,Benign neoplasm of body of tongue,116680003,舌体良性肿瘤
9,D38.5,其他呼吸器官动态未定或动态未知的肿瘤,source,95433000,Disorder of nasal septum (disorder),116680003,鼻中隔紊乱


#### 查询 relation type

In [60]:
with open('snomed_relationship_type.txt') as fget:
    lines = [line.strip() for line in fget.readlines()]

In [61]:
relationship_type = [re.search(pattern=r'(\d*)\ \|(.*)\|', string=line).groups() for line in lines]

In [62]:
relationship_type = {pair[0]: pair[1] for pair in relationship_type}

In [63]:
def look_type(l, d=relationship_type):
    return [d[str(item)] for item in l]

In [90]:
data_df['relationship_type'] = look_type(data_df['relation_type_id'])

In [93]:
output = data_df[['icd_id','icd_name','another_sctid','another_sct_term_cn','another_sct_term','relationship_type','icd_where',]]

In [95]:
output.to_csv('Snomed CT 中疾病相关关系（仅包含有term可查的）.tsv', index=None, sep='\t')

In [8]:
df2 = pd.read_table('Snomed_CT_relation.tsv')

In [86]:
df2.sort_values(by='relationship_type').to_csv('Snomed CT 中疾病相关关系（仅包含有term可查的）.tsv', index=None, sep='\t')

In [29]:
relationship_types = list(df2['relationship_type'])
counts = [{'ty': ty, 'count': relationship_types.count(ty)} for ty in set(relationship_types)]
pd.DataFrame(counts)[['ty', 'count']]

,ty,count
0,Interprets,114
1,Associated with,14
2,Associated morphology,309
3,Associated procedure,14
4,Has focus,13
5,Causative agent,87
6,Associated finding,51
7,Due to,214
8,After,76
9,Is a (attribute),3047


### 两个概念均有对应的ICD编码的关系

In [ ]:
rela

In [47]:
condition_df = rela.isin({'sourceId': set(icd2sct['conceptId']), 'destinationId': set(icd2sct['conceptId'])})

In [51]:
both_icd = rela[condition_df['sourceId']][condition_df['destinationId']]

C:\Users\homewin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [57]:
for row in both_icd.iterrows():
    assert row[1]['sourceId'] in icd_id_set and row[1]['destinationId'] in icd_id_set

In [58]:
both_icd

,id,effectiveTime,active,moduleId,sourceId,destinationId,relationshipGroup,typeId,characteristicTypeId,modifierId
26,5442029,20190731,0,900000000000207008,10273003,57054005,0,116680003,900000000000011006,900000000000451002
35,7048022,20190731,1,900000000000207008,1034003,128321003,0,116680003,900000000000011006,900000000000451002
137,17822029,20190731,0,900000000000207008,186150001,55184003,0,116680003,900000000000011006,900000000000451002
139,18161022,20190731,0,900000000000207008,187094001,42386007,0,116680003,900000000000011006,900000000000451002
143,19062020,20190731,0,900000000000207008,105649009,62005008,0,116680003,900000000000011006,900000000000451002
147,19661028,20190731,0,900000000000207008,192743008,15758002,0,116680003,900000000000011006,900000000000451002
149,19745024,20190731,0,900000000000207008,105986008,8447006,0,116680003,900000000000011006,900000000000451002
152,19845027,20190731,0,900000000000207008,193504001,17657000,0,116680003,900000000000011006,900000000000451002
153,19846026,20190731,0,900000000000207008,193504001,62585004,0,116680003,900000000000011006,900000000000451002
156,20289029,20190731,1,900000000000207008,195289005,128321003,0,116680003,900000000000011006,900000000000451002


In [87]:
both_icd_result = [{
    'source_icd_id': list(icd2sct[icd2sct['conceptId']==row[1]['sourceId']]['亚目编码'])[0],
    'source_icd_name': list(icd2sct[icd2sct['conceptId']==row[1]['sourceId']]['亚目名称'])[0], 
    'destination_icd_id': list(icd2sct[icd2sct['conceptId']==row[1]['destinationId']]['亚目编码'])[0], 
    'destination_icd_name': list(icd2sct[icd2sct['conceptId']==row[1]['destinationId']]['亚目名称'])[0], 
    'relationship_type': relationship_type[str(row[1]['typeId'])], 
    'relationship_id': row[1]['id']
} for row in both_icd.iterrows() if list(icd2sct[icd2sct['conceptId']==row[1]['sourceId']]['亚目编码'])[0] != list(icd2sct[icd2sct['conceptId']==row[1]['destinationId']]['亚目编码'])[0]]

In [88]:
both_icd_result = pd.DataFrame(both_icd_result)[['source_icd_id', 'source_icd_name', 'destination_icd_id', 'destination_icd_name', 'relationship_type', 'relationship_id', ]]

In [96]:
both_icd_result = both_icd_result.drop_duplicates(['source_icd_id', 'destination_icd_id'])

In [97]:
both_icd_result.sort_values(by='relationship_type')

,source_icd_id,source_icd_name,destination_icd_id,destination_icd_name,relationship_type,relationship_id
3068,T91.8,颈部和躯干其他特指损伤的后遗症,T14.9,未特指的损伤,After,3239212022
5980,G91.1,梗阻性脑积水,T14.9,未特指的损伤,After,7171248021
5979,G91.0,交通性脑积水,T14.9,未特指的损伤,After,7171247027
5978,G91.3,未特指的创伤后脑积水,T14.9,未特指的损伤,After,7171210027
5965,P11.2,产伤引起的未特指的脑损害,G93.1,"缺氧性脑损害,不可归类在他处者",After,7170209028
1514,T90.9,头部未特指损伤的后遗症,S09.9,头部未特指的损伤,After,2582109021
1508,E23.0,垂体功能减退症,P15.9,未特指的产伤,After,2580020028
1506,I05.8,其他二尖瓣疾病,A49.1,未特指部位的链球菌感染,After,2578288021
1505,L90.5,皮肤瘢痕情况和纤维化,L70.9,未特指的痤疮,After,2577014021
1504,T94.1,"损伤后遗症,未按身体部位特指者",T14.4,身体未特指部位的神经损伤,After,2576518021


In [98]:
both_icd_result.sort_values(by='relationship_type').to_csv('Snomed CT 中 ICD_terms 相关关系.tsv', index=None, sep='\t')

In [99]:
relationship_types = list(both_icd_result['relationship_type'])
counts = [{'ty': ty, 'count': relationship_types.count(ty)} for ty in set(relationship_types)]
pd.DataFrame(counts)[['ty', 'count']]

,ty,count
0,Associated with,243
1,After,231
2,Due to,981
3,Is a (attribute),5633
4,Associated finding,44


In [100]:
both_icd_result[both_icd_result['relationship_type'] == 'Due to']

,source_icd_id,source_icd_name,destination_icd_id,destination_icd_name,relationship_type,relationship_id
1435,E14.3,NaN,E14.9,未特指的糖尿病不伴有并发症,Due to,2568574020
1440,E14.5,NaN,E14.9,未特指的糖尿病不伴有并发症,Due to,2568895022
1453,G12.2,运动神经元病,D48.9,未特指的动态未定或动态未知的肿瘤,Due to,2571445020
1456,G25.3,肌阵挛,G93.1,"缺氧性脑损害,不可归类在他处者",Due to,2571452022
1457,E53.9,未特指的维生素B缺乏病,E56.9,未特指的维生素缺乏病,Due to,2571513029
1458,E14.4,NaN,E14.9,未特指的糖尿病不伴有并发症,Due to,2571514024
1471,H53.0,失用性弱视,H52.3,屈光参差和影像不等,Due to,2572883025
1473,Q85.1,结节性硬化症,Q85.9,未特指的斑痣性错构瘤病,Due to,2573623021
1479,D51.1,选择性维生素B12吸收不良伴有蛋白尿引起的维生素B12缺乏性贫血,E53.8,其他特指的B族维生素缺乏病,Due to,2573993029
1481,E14.2,NaN,E14.9,未特指的糖尿病不伴有并发症,Due to,2574706028


In [101]:
both_icd_result[both_icd_result['relationship_type'] == 'After']

,source_icd_id,source_icd_name,destination_icd_id,destination_icd_name,relationship_type,relationship_id
1428,B90.9,呼吸道结核和未特指结核的后遗症,A16.9,"未特指的呼吸道结核,未提及细菌学或组织学的证实",After,2568360028
1429,B90.0,中枢神经系统结核的后遗症,A16.9,"未特指的呼吸道结核,未提及细菌学或组织学的证实",After,2568361029
1430,B90.1,泌尿生殖系结核的后遗症,A16.9,"未特指的呼吸道结核,未提及细菌学或组织学的证实",After,2568362020
1431,B90.2,骨和关节结核的后遗症,A16.9,"未特指的呼吸道结核,未提及细菌学或组织学的证实",After,2568363026
1432,B94.2,病毒性肝炎的后遗症,B19.9,"未特指的病毒性肝炎,不伴有肝昏迷",After,2568365022
1433,E64.9,未特指的营养缺乏后遗症,E63.9,未特指的营养缺乏,After,2568495029
1434,E64.1,维生素A缺乏后遗症,E50.9,未特指的维生素A缺乏病,After,2568497021
1436,I01.2,急性风湿性心肌炎,A49.1,未特指部位的链球菌感染,After,2568652020
1437,I09.0,风湿性心肌炎,A49.1,未特指部位的链球菌感染,After,2568653026
1438,I69.8,其他和未特指的脑血管病后遗症,I67.9,未特指的脑血管病,After,2568661020


In [102]:
both_icd_result[both_icd_result['relationship_type'] == 'Associated finding']

,source_icd_id,source_icd_name,destination_icd_id,destination_icd_name,relationship_type,relationship_id
3176,O35.0,为胎儿(可疑)中枢神经系统畸形给予的孕产妇医疗\t,Q03.9,未特指的先天性脑积水,Associated finding,3419499021
3182,O35.0,为胎儿(可疑)中枢神经系统畸形给予的孕产妇医疗\t,Q05.9,未特指的脊柱裂,Associated finding,3437473025
3939,Z80.4,生殖器官恶性肿瘤家族史,D40.9,男性生殖器官未特指的动态未定或动态未知的肿瘤,Associated finding,4739112028
3980,Z85.4,生殖器官恶性肿瘤个人史,C63.9,未特指的男性生殖器官恶性肿瘤,Associated finding,4742430027
4056,Z92.4,"大手术个人史,不可归类在他处者",Q24.9,未特指的先天性心脏畸形,Associated finding,4776861021
5225,Z87.5,妊娠、分娩和产褥期并发症个人史,P08.0,特大婴儿,Associated finding,6490664023
7493,Z86.6,神经系统和感觉器官疾病个人史,H50.9,未特指的斜视,Associated finding,11225968026
7507,Z86.2,血液和造血器官疾病和某些涉及免疫机制的疾患个人史,D69.6,未特指的血小板减少,Associated finding,11226121026
9896,Z86.0,其他肿瘤个人史,D36.1,周围神经和自主神经系统良性肿瘤,Associated finding,11418430021
10121,Z86.6,神经系统和感觉器官疾病个人史,H47.0,"视神经疾患,不可归类在他处者",Associated finding,11433832029
